# FIN383 GBA JAN22

Group Number: 03

Group Members:
* Ernest Pek Jun Wei (B1981095) (Leader)
* Darren Lim Jun Yang (H1882523)
* Tang Kai Ming Clement (H1970491)
* Gwendalyn Sim (Z2081803)

T Group: T02

Submission date: 2022-02-22

#### Code and Question 1(b) to (e) by Gwendalyn & Darren

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
md = "ADJ_CLOSE.zip"                     #zipped market data from 1990 to 2020
dsi_range = ["1991-01-01", "2020-12-31"] #DSI is calculated from 1991 to 2020
sim_range = ["1990-01-01", "2010-12-31"] #simulation uses data from 1990 to 2010
val_range = ["2010-01-01", "2020-12-31"] #validation uses data from 2010 to 2020
FG = {True:"Fearful", False:"Greedy"}    #used to print DSI, MSI boolean as string

In [3]:
df = pd.read_csv((md), parse_dates = ['Date'], index_col = ['Date']) #importing the "ADJ_CLOSE.zip" into Python and setting the 'Date' as the DataFrame's index
print("\nInitial Dataset:\n")
display(df)


Initial Dataset:



,vix,s_0000,s_0001,s_0002,s_0003,s_0004,s_0005,s_0006,s_0007,s_0008,...,s_2579,s_2580,s_2581,s_2582,s_2583,s_2584,s_2585,s_2586,s_2587,s_2588
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,17.24,NaN,13.389421,NaN,NaN,NaN,NaN,NaN,0.266812,NaN,...,NaN,1.950358,NaN,7.253997,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,18.19,NaN,13.588601,NaN,NaN,NaN,NaN,NaN,0.268603,NaN,...,NaN,1.985185,NaN,7.253997,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,19.22,NaN,13.610730,NaN,NaN,NaN,NaN,NaN,0.269499,NaN,...,NaN,1.985185,NaN,7.188052,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,20.11,NaN,13.477942,NaN,NaN,NaN,NaN,NaN,0.270394,NaN,...,NaN,1.985185,NaN,7.188052,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,20.26,NaN,13.477942,NaN,NaN,NaN,NaN,NaN,0.272185,NaN,...,NaN,1.985185,NaN,7.385889,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,23.31,116.631310,22.171579,15.890000,16.577030,9.00,65.023491,157.495850,130.347580,27.481012,...,19.870001,42.675430,2.90,8.850000,9.93,NaN,0.226,207.470001,158.974014,36.650002
2020-12-24,21.53,116.641243,21.912146,15.660000,16.606722,8.77,65.292725,158.870193,131.352829,27.813406,...,20.180000,42.508686,2.88,8.810000,9.91,NaN,0.229,205.270004,159.839264,36.009998
2020-12-28,21.70,117.158287,22.191536,16.059999,16.200956,8.93,66.429459,157.011383,136.050766,28.272888,...,19.959999,42.528305,2.69,8.760000,9.73,NaN,0.251,199.369995,161.500122,36.709999


In [4]:
def hvm(date): #defining the historical median as the function hvm
    t = pd.Timestamp(date)
    d = np.timedelta64(1,'D')             #for increments or decrements of one day
    m = np.timedelta64(1,'M')             #for increments or decrements of one month
    y = np.timedelta64(1,'Y')             #for increments or decrements of one year
    A = (t-y+d).replace(day=1) - d        #start date of the 12-months historical median 
    B = t.replace(day=1) - d              #end date of the 12-months historical median
    hvm12 = df.loc[A:B, "vix"].median()   
    return hvm12

hvmdic = {} #initiating an empty dictionary called hvmdic

for year in range(1991,2021): #iterating from years 1991 to 2020
    for month in range(1,13): #iterating through the months January to December
        date = str(year) + '-' + str(month) 
        date_time_obj = datetime.datetime.strptime(date, '%Y-%m')
        hvmdic[date_time_obj] = hvm(str(year)+"-"+str(month)+"-1") #storing the first day of the month and its respective HVM as the key-value pair in the hvmdic dictionary
        
df2 = pd.DataFrame.from_dict(hvmdic, orient='index', columns=['HVM12']) #creating a DataFrame from the hvmdic dictionary
display(df2)

,HVM12
1991-01-01,22.570
1991-02-01,22.740
1991-03-01,22.290
1991-04-01,22.290
1991-05-01,21.790
...,...
2020-08-01,18.980
2020-09-01,22.245
2020-10-01,24.840
2020-11-01,26.405


In [5]:
new_index = df.index                         #creating a new DataFrame to match the index of HVM with the orignal dataset
df3 = df2.reindex(new_index, method='ffill') #forward filling the rest of the 'NaN' values
display(df3)

,HVM12
Date,
1990-01-02,NaN
1990-01-03,NaN
1990-01-04,NaN
1990-01-05,NaN
1990-01-08,NaN
...,...
2020-12-23,26.77
2020-12-24,26.77
2020-12-28,26.77


In [6]:
new_df = pd.concat([df, df3], axis=1) #concatenating df2 with the main DataFrame
display(new_df)

,vix,s_0000,s_0001,s_0002,s_0003,s_0004,s_0005,s_0006,s_0007,s_0008,...,s_2580,s_2581,s_2582,s_2583,s_2584,s_2585,s_2586,s_2587,s_2588,HVM12
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,17.24,NaN,13.389421,NaN,NaN,NaN,NaN,NaN,0.266812,NaN,...,1.950358,NaN,7.253997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,18.19,NaN,13.588601,NaN,NaN,NaN,NaN,NaN,0.268603,NaN,...,1.985185,NaN,7.253997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,19.22,NaN,13.610730,NaN,NaN,NaN,NaN,NaN,0.269499,NaN,...,1.985185,NaN,7.188052,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,20.11,NaN,13.477942,NaN,NaN,NaN,NaN,NaN,0.270394,NaN,...,1.985185,NaN,7.188052,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,20.26,NaN,13.477942,NaN,NaN,NaN,NaN,NaN,0.272185,NaN,...,1.985185,NaN,7.385889,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,23.31,116.631310,22.171579,15.890000,16.577030,9.00,65.023491,157.495850,130.347580,27.481012,...,42.675430,2.90,8.850000,9.93,NaN,0.226,207.470001,158.974014,36.650002,26.77
2020-12-24,21.53,116.641243,21.912146,15.660000,16.606722,8.77,65.292725,158.870193,131.352829,27.813406,...,42.508686,2.88,8.810000,9.91,NaN,0.229,205.270004,159.839264,36.009998,26.77
2020-12-28,21.70,117.158287,22.191536,16.059999,16.200956,8.93,66.429459,157.011383,136.050766,28.272888,...,42.528305,2.69,8.760000,9.73,NaN,0.251,199.369995,161.500122,36.709999,26.77


In [7]:
new_df.loc[new_df['vix'] <= new_df['HVM12'], 'DSI'] = 'Greedy' #if calculated HVM is smaller or equal to VIX, assign DSI as 'Greedy'
new_df.loc[new_df['vix'] > new_df['HVM12'], 'DSI'] = 'Fearful' #if calculated HVM is greater than VIX, assign DSI as 'Fearful'

In [8]:
subset_Fear = new_df[new_df["DSI"] == 'Fearful'] #creating a new DataFrame where the DSI is 'Fearful'
subset_Fear = subset_Fear[['DSI']].groupby(pd.Grouper(level='Date',freq='M')).count().rename(columns={"DSI": "Fear"}) #grouping and summing the number of 'Fearful' per month
display(subset_Fear)

,Fear
Date,
1991-01-31,19
1991-02-28,3
1991-03-31,0
1991-04-30,0
1991-05-31,0
...,...
2020-07-31,22
2020-08-31,21
2020-09-30,21


In [9]:
dsi_compare = pd.concat([subset_Greed, subset_Fear], axis=1) #concatenating both subsets for comparison
display(dsi_compare)
dsi_compare.at['2020-12-31','Fear'] = 0 #changing the count of 'Fear' from 'Nan' to 0
display(dsi_compare)

NameError: name 'subset_Greed' is not defined

In [ ]:
# creating a new column in the DataFrame for no. of days in the month
dsi_compare['No. of Days'] = dsi_compare.apply(lambda row: row.Greed + row.Fear, axis=1) 
#creating a new column in the DataFrame for the month
dsi_compare["Month"] = dsi_compare.index.month

In [ ]:
dsi_compare.loc[dsi_compare['Greed'] >= dsi_compare['Fear'], 'MSI'] = 'Greedy' #if there are more 'Greedy' compared to 'Fearful', assign MSI as 'Greedy'
dsi_compare.loc[dsi_compare['Fear'] > dsi_compare['Greed'], 'MSI'] = 'Fearful' #if there are more 'Fearful' compared to 'Greedy', assign MSI as 'Fearful'
display(dsi_compare)

In [ ]:
cal = USFederalHolidayCalendar()

def get_business_day(date):                                  #defining a function to obtain the next business day by excluding weekends or holidays
    while date.isoweekday() > 5 or date in cal.holidays():
        date += datetime.timedelta(days=1)
    return date

ts1 = pd.DataFrame(dsi_compare[['MSI']])                     #copying the 'MSI' DataFrame into a new DataFrame 'ts1'
ts1 = ts1.shift(1, freq='D').rename(columns={"MSI": "TS"})   #shifting the index of ts by one day from the MSI date to first trading day of each month
ts1.loc[ts1['TS'] == 'Fearful'] = 'Buy'                      #Renaming the term 'Fearful' to 'Buy'
ts1.loc[ts1['TS'] == 'Greedy'] = 'Sell'                      #Renaming the term 'Greedy' to 'Sell'

first_bizday_of_month = []                                   #initialising an empty list for the first trading day of the month

for index in ts1.index:                                      #interating through the index of the Trading Signals DataFrame
    fbdm = get_business_day(index)                           #assigning the date of the business day obtained to the variable, fbdm
    first_bizday_of_month.append(fbdm)                       #storing the date to the list

ts1['Date'] = first_bizday_of_month                          #creating a column 'Date' for the Trading Signals DataFrame with the first business days
ts1 = ts1.set_index('Date')                                  #setting the 'Date' columnas the index of the DataFrame
display(ts1)

In [ ]:
msi = dsi_compare[['MSI']]
df4 = pd.concat([new_df, msi, ts1], axis=1) #concatenating MSI and TS with the main DataFrame
display(df4)

### 1b)

In [ ]:
ltd_mar_2008_dsi = df4.loc['2018-03-28', 'DSI']
ltd_dec_2020_dsi = df4.loc['2020-12-30', 'DSI']

print(f'The DSI of the last trading day of Mar 2008 is {ltd_mar_2008_dsi}.')
print(f'The DSI of the last trading day of Mar 2008 is {ltd_dec_2020_dsi}.')

### 1c)

In [ ]:
Fearful_percent = []
for mth in range(1,13):
    fear_percent = ((dsi_compare.groupby('Month')['Fear'].sum()[mth]) / (dsi_compare.groupby('Month')['No. of Days'].sum()[mth])).round(4)
    Fearful_percent.append(fear_percent)

fearful_data = {'Month' : ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
               , '%_fearful_days' : [0.461, 0.4783, 0.4824, 0.3961, 0.4385, 0.4649, 0.3758, 0.4449, 0.468, 0.5973, 0.4837, 0.4518]}

fearful_df = pd.DataFrame(fearful_data)

fearful_df['month'] = pd.DatetimeIndex(pd.to_datetime(fearful_df['Month'], format='%b')).month
fearful_df = fearful_df.set_index('month').sort_index()

display(fearful_df)

fearful_df.plot(kind="bar")

### 1d)

In [ ]:
print('The most "fearful" month is October, while the least "fearful" month is July.')

### 1e)

In [ ]:
mar_2013_msi = df4.loc['2013-03-31', 'MSI']
jun_2013_msi = df4.loc['2013-06-30', 'MSI']
sep_2013_msi = df4.loc['2013-09-30', 'MSI']
dec_2013_msi = df4.loc['2013-12-31', 'MSI']

print(f'The computed MSI of Mar 2013 is {mar_2013_msi}.')
print(f'The computed MSI of Jun 2013 is {jun_2013_msi}.')
print(f'The computed MSI of Sep 2013 is {sep_2013_msi}.')
print(f'The computed MSI of Dec 2013 is {dec_2013_msi}.')

### Code and Question 1(f) to (i) by Ernest & Syakir

### 1f)

In [ ]:
TS_1991_2010 = df4.loc['1991-01-01':'2010-12-31','TS'] #selecting the rows for period 1991 to 2010
print(f'The total trading signals for period 1991 to 2010:')
TS_1991_2010.value_counts()

In [ ]:
TS_2011_2020 = df4.loc['2011-01-01':'2020-12-31','TS'] #selecting the rows for period 2011 to 2020
print("The total trading signals for period 2011 to 2020:")
TS_2011_2020.value_counts()

In [ ]:
df5 = df['1991-01':'2010-12'].iloc[:, 1:] #creating a new DataFrame that is filtered from 1991 to 2010
display(df5)
df5.dropna(axis=1, how='all')

In [ ]:
ts2 = ts1.loc['1991-01-01':'2010-12-31']        #creating a new Trading Signals DataFrame that is filtered from 1991 to 2010
ts2 = ts2.reset_index()                         #resetting the index
ts2['Date'] = ts2['Date'].dt.strftime('%Y-%m')  #reformating the Date of the DataFrame
ts2['Date'] = ts2['Date'].astype(str)           #converting the datatype of the Date from timestamp to string
display(ts2)

In [ ]:
stock_names = df5.columns.values.tolist() #obtaining a list of all the names of the stocks in the dataset
P_n_L = []                                #initialising an empty list for Profit & Loss

for s in stock_names:                     #iterating through the stocks in the dataset
    profit = 0                            #initialising the profit of the stock that begins from 0
    initial_ts = 'Sell'                   #initialising the first trading signal as 'Sell' to prepare for the next Trading Signal
    for index, column in ts2.iterrows():  #iterating through the rows and columns of the ts2 DataFrame
        if column['TS'] == 'Buy' and initial_ts == 'Sell' and df5[s][column['Date']][0] != 'NaN':
            qty = 10000/df5[s][column['Date']][0]
            initial_ts = 'Buy'            #changing the status of the Trading Signal
        if column['TS'] == 'Sell' and initial_ts == 'Buy' and df5[s][column['Date']][0] != 'NaN':
            profit += df5[s][column['Date']][0] * qty #cumulative summing the profit for the stock
            initial_ts = 'Sell'           #changing the status of the Trading Signal
    P_n_L.append((s, profit))

In [ ]:
P_n_L = [x for x in P_n_L if not math.isnan(x[1])]            # removing null values from the list of Profit & Loss
t5 = sorted(P_n_L, key = lambda x: x[1], reverse = True)[:5]  #sorting the Profit & Loss list in descending orderand showing the top 5 records  
t5_s = [stock[0] for stock in t5]                             #to identify only the stock names since t5 also gives us the respective profits 
print(f'From 1991 to 2010, the top 5 profitable stocks using the strategy Simulation are {t5_s}.')

### 1h)

In [ ]:
ts3 = ts1.loc['2011-01-01':'2020-12-31']       #creating a new Trading Signals DataFrame that is filtered from 2011 to 2020
ts3 = ts3.reset_index()                        #resetting the index
ts3['Date'] = ts3['Date'].dt.strftime('%Y-%m') #reformating the Date of the DataFrame
ts3['Date'] = ts3['Date'].astype(str)          #converting the datatype of the Date from timestamp to string
display(ts3)               

df6 = df['2011-02':'2020-12'].iloc[:, 1:] #creating a new DataFrame that is filtered from 2011 to 2020
df6.dropna(axis=1, how='all')

In [ ]:
#same 5 stocks using the strategy Validation from year 2011 to year 2020
top_5 = [stock[0] for stock in t5]       #creating a list of the names of the top 5 stocks
P_n_L2 = []                              #initialising an empty list for Profit & Loss

for s in top_5:                          #iterating through the names of the top 5 stocks
    profit = 0                           #initialising the profit of the stock that begins from 0
    initial_ts = 'Sell'                  #initialising the first trading signal as 'Sell' to prepare for the next Trading Signal
    for index, column in ts3.iterrows(): #iterating through the rows and columns of the ts3 DataFrame
        if column['TS'] == 'Buy' and initial_ts == 'Sell' and df6[s][column['Date']][0] != 'NaN':
            qty = 10000/df6[s][column['Date']][0]
            initial_ts = 'Buy'           #changing the status of the Trading Signal
        if column['TS'] == 'Sell' and initial_ts == 'Buy' and df6[s][column['Date']][0] != 'NaN':
            profit += df6[s][column['Date']][0] * qty
            initial_ts = 'Sell'          #changing the status of the Trading Signal
    P_n_L2.append((s, profit))

In [ ]:
print(f"The profits of the same 5 stocks according to strategy Validation from 2011 to 2020 are {P_n_L2[0]},{P_n_L2[1]},{P_n_L2[2]},{P_n_L2[3]} and {P_n_L2[4]}.")

### 1i)

In [ ]:
t50_stock = sorted(P_n_L, key = lambda x: x[1], reverse = True)[:50] #selecting the top 50 stocks from the strategy Simulation
t50_name = [stock[0] for stock in t50_stock]                         #extracting the names of the top 50 stocks

In [ ]:
#using the top 50 stocks from strategy Simulation and running them through the strategy validation
P_n_L50 = []                                #initialising an empty list for Profit & Loss                          
for s in t50_name:                          #iterating through the names of the top 50 stocks                                                 
    profit = 0                              #initialising the profit of the stock that begins from 0
    initial_ts = 'Sell'                     #initialising the first trading signal as 'Sell' to prepare for the next Trading Signal
    for index, column in ts3.iterrows():    #iterating through the rows and columns of the ts3 DataFrame
        if column['TS'] == 'Buy' and initial_ts == 'Sell' and df6[s][column['Date']][0] != 'NaN':
            qty = 10000/df6[s][column['Date']][0]
            initial_ts = 'Buy'              #changing the status of the Trading Signal
        if column['TS'] == 'Sell' and initial_ts == 'Buy' and df6[s][column['Date']][0] != 'NaN':
            profit += df6[s][column['Date']][0] * qty
            initial_ts = 'Sell'             #changing the status of the Trading Signal
    P_n_L50.append((s, profit))    

In [ ]:
display(t50_stock) #Showing the top 50 stocks and their profits using the strategy Simulation

In [ ]:
P_n_L50_sorted = sorted(P_n_L50, key = lambda x: x[1], reverse = True)[:50] #sorting the top 50 stocks in descending order

In [ ]:
display(P_n_L50_sorted) #Showing the top 50 stocks and their profits using the strategy Simulation

The top 50 stocks' profitability when using the strategy Validation are lower when compared to when the strategy Simulation was used.

## Conclusion ( Question (j) )

The group noted that, based on the analysis of the dataset, the strategy "buy on fear; sell on greed" is profitable. This strategy has worked for decades and has generated high returns as shown in our P/L data. However, just relying on the Fear and/or Greed Index is never sufficient information when investing. Investors should consider all available information before they decide whether to buy or sell a stock.